# Import from Database and Analysis
This notebook imports shared Craigslist listings from the database on Istanbul for initial data exploration and analysis.

In [ ]:
#import packages
import pandas as pd
import numpy as np
import re as re
import json    # library for working with JSON-formatted text strings
import requests  # library for accessing content from web URLs
import pprint  # library for making Python data structures readable
import psycopg2
import matplotlib.font_manager as fm
import matplotlib.pyplot as plt
import plotly as py
from plotly.graph_objs import *
pp = pprint.PrettyPrinter()
pd.options.mode.chained_assignment = None #disables warnings for editing copy of a dataframe
pd.set_option('display.float_format', lambda x: '%.3f' % x) #describe() vars are not in scientific notation
pd.set_option('max_columns', 30)
%matplotlib inline

In [ ]:
# Defining font styles for graphics
family = 'Arial'
title_font = fm.FontProperties(family=family, style='normal', size=18, weight='normal', stretch='normal')
label_font = fm.FontProperties(family=family, style='normal', size=16, weight='normal', stretch='normal')
ticks_font = fm.FontProperties(family=family, style='normal', size=14, weight='normal', stretch='normal')

In [ ]:
# Creating function for defining colors for graphics
def get_colors(cmap, n, start=0., stop=1., alpha=1., reverse=False):
    '''return n-length list of rgba colors from the passed colormap name and alpha,
       limit extent by start/stop values and reverse list order if flag is true'''
    colors = [cm.get_cmap(cmap)(x) for x in np.linspace(start, stop, n)]
    colors = [(r, g, b, alpha) for r, g, b, _ in colors]
    return list(reversed(colors)) if reverse else colors

In [ ]:
# Read in credentials from private settings file
with open('settings.json') as settings_file:    
    settings = json.load(settings_file)

In [ ]:
#try to connect to database on local machine
dbname = settings['dbname']
user = settings['user']
host = settings['host']
password = settings['password']

conn_str = "dbname = {0} user = {1} host = {2} password = {3}".format(dbname, user, host, password)

try:
    conn = psycopg2.connect(conn_str)
except:
    print ("I am unable to connect to the database")

In [ ]:
# example dataframe with all the listings (as of 04/24, about 23k listings)
df = pd.read_sql_query("select * from shared_listings;",con=conn)

In [ ]:
df.shape

### Example SQL pulls

In [ ]:
# example syntax for pulling just those from newyork domain
df_nyc = pd.read_sql_query("select * from shared_listings where region = 'newyork';",con=conn)

In [ ]:
# example syntax for pulling just those on a specific date (April 22)
df_april25 = pd.read_sql_query("select * from shared_listings where dt >= '2017-04-25' AND dt < '2017-04-23';",con=conn)

In [ ]:
# example syntax for pulling just those on a specific date (April 22)
df_april25 = pd.read_sql_query("select * from shared_listings where dt >= '2017-04-25';",con=conn)

In [ ]:
df_april24 = pd.read_sql_query("select * from shared_listings where dt >= '2017-04-24'AND dt < '2017-04-25';",con=conn)

In [ ]:
df_april23 = pd.read_sql_query("select * from shared_listings where dt >= '2017-04-23'AND dt < '2017-04-24';",con=conn)

In [ ]:
#close database connection (no longer needed)
conn.close()

### De-Duplication

In [ ]:
df.loc[df.lat == 99, 'lat'] = 0
df.loc[df.lng == 99, 'lng'] = 0
df.loc[df.sqft == 0, 'sqft'] = np.nan

#For any duplicate post, we want to keep the version with the most information. Therefore, assign a score to each post and 
#give one point for square footage, lat, long and price

df['price_exists'] = df['rent']>0
df['sqft_exists'] = df['sqft']>0
df['lat_exists'] = df['lat']>0
df['lng_exists'] = df['lng']<0
df['score'] = df[['price_exists','sqft_exists','lat_exists','lng_exists']].astype(bool).sum(axis=1)

In [ ]:
#Sort rows by score
df = df.sort_values(by='score',ascending=False)

In [ ]:
dedupe1 = pd.DataFrame(df.drop_duplicates(subset='pid', inplace=False))

In [ ]:
dedupe1.shape

In [ ]:
dedupe2 = pd.DataFrame(dedupe1.drop_duplicates(subset='body_text', inplace=False))

In [ ]:
dedupe2.shape

### Quick bar chart to see impact of deduplication on sample size

In [ ]:
y = [len(df),len(dedupe1), len(dedupe2)]
labels = ['original', 'dedupe_pid', 'dedupe_body']
x = [1,2,3]

In [ ]:
plt.figure(figsize=(10,6) )
plt.suptitle('Deduplication Sample Size')
plt.xlabel('Deduplication Phase')
plt.ylabel('Number of Listings')
plt.xticks(x, labels)
ax = plt.bar(x, y, alpha=.4, color='cyan', align='center')
plt.show()

#### Code for plotting same histo w/Plotly

In [ ]:
#For initializing offline mode. Not working though...
#py.offline.init_notebook_mode() 

In [ ]:
data = [Bar(x=['original','dedupe_pid','dedupe_body'], y=[len(df),len(dedupe1), len(dedupe2)])]
py.offline.plot(data)

### More Filtering

In [ ]:
# Filtering out listings that don't include rent price
unique_wprice = dedupe2[dedupe2['rent'] > 0]

In [ ]:
unique_wprice.shape

In [ ]:
# in this cell, define the values by which we will filter the 3 columns. This will vary depending on the sample we're looking at. 
upper_percentile = 0.997
lower_percentile = 0.08

# how many rows would be within the upper and lower percentiles?
upper = int(len(unique_wprice) * upper_percentile)
lower = int(len(unique_wprice) * lower_percentile)

# get the rent values at the upper and lower percentiles
rent_sorted = unique_wprice['rent'].sort_values(ascending=True, inplace=False)
upper_rent = rent_sorted.iloc[upper]
lower_rent = rent_sorted.iloc[lower]

# get the sqft values at the upper and lower percentiles
sqft_sorted = unique_wprice['sqft'].sort_values(ascending=True, inplace=False)
upper_sqft = sqft_sorted.iloc[upper]
lower_sqft = sqft_sorted.iloc[lower]

print('valid rent range:', [lower_rent, upper_rent])
print('valid sqft range:', [lower_sqft, upper_sqft])

In [ ]:
#Filtering out rows with unreasonable rent prices 
rent_mask = (unique_wprice['rent'] > lower_rent) & (unique_wprice['rent'] < upper_rent)

In [ ]:
filtered_listings = pd.DataFrame(unique_wprice[rent_mask])

In [ ]:
filtered_listings.shape

In [ ]:
filtered_listings.dtypes

In [ ]:
#Creating dataframe that only includes shared listings that are private rooms
filtered_private_only = filtered_listings[filtered_listings['private_room']==True]

filtered_private_only.shape

### Full Filtered Database Charts

In [ ]:
listings_ratios = pd.DataFrame()

#Number of total shared listings for reach region before deduplication or filtering
listings_ratios['all_shared_listings'] = df['region'].value_counts()

listings_ratios['deduplicated_pid'] = dedupe1['region'].value_counts()

listings_ratios['deduplicated_text'] = dedupe2['region'].value_counts()

listings_ratios['duplicate_listings'] = listings_ratios['all_shared_listings'] - listings_ratios['deduplicated_text'] 

listings_ratios['duplicate_ratio'] = listings_ratios['duplicate_listings']/listings_ratios['all_shared_listings']

listings_ratios['unique_ratio'] = listings_ratios['deduplicated_text']/listings_ratios['all_shared_listings']

In [ ]:
listings_ratios.head()

In [ ]:
# plot the ratios of unique and duplicate listings, by region. This is before dropping observations without lat-lngs!

countdata = listings_ratios.sort_values(by='all_shared_listings', ascending=False)[['deduplicated_text','duplicate_listings']].head(20)
countdata.columns = ['Unique Listings', 'Duplicate Listings']
ax = countdata.plot(kind='bar',
                    stacked=True,
                    figsize=[10, 6], 
                    width=0.6, 
                    alpha=0.5, 
                    color=['b','m'],
                    edgecolor='k',
                    grid=False)

ax.yaxis.grid(True)
ax.set_xticks(range(0, len(countdata)))
ax.set_xticklabels(countdata.index, rotation=40, rotation_mode='anchor', ha='right', fontproperties=ticks_font)
for label in ax.get_yticklabels():
        label.set_fontproperties(ticks_font)
ax.set_title('Unique and Duplicate Shared Rental Listings, by Region', fontproperties=title_font)
ax.set_xlabel('', fontproperties=label_font)
ax.set_ylabel('Total number of listings', fontproperties=label_font)        

#save_fig(plt.gcf(), 'count_unique_duplicate_listings.png')
plt.show()

## Spatial Join for Census Tracts

In [ ]:
import geopandas as gpd

from geopandas import GeoDataFrame
from shapely.geometry import Point
from geopy.distance import great_circle

from scipy import ndimage

import matplotlib.pylab as pylab
import matplotlib.pyplot as plt
pylab.rcParams['figure.figsize'] = 10, 8
import fiona

### California Regions

In [ ]:
california = gpd.read_file('ca_census_shapefile//cb_2015_06_tract_500k.shp')
print(type(california))
print(california.crs)

In [ ]:
#Use California CRS
# California.crs = {'init' :'epsg:2227'}
#http://spatialreference.org/ref/epsg/2227/
california_crs = {'init': 'epsg:2227'}
california = california.to_crs(california_crs)
print(california.crs)

In [ ]:
sfbay = filtered_private_only[filtered_private_only['region']=='sfbay']

In [ ]:
sfbay = sfbay[sfbay.lat != 0]

In [ ]:
los_angeles = filtered_private_only[filtered_private_only['region']=='losangeles']

In [ ]:
los_angeles = los_angeles[los_angeles.lat != 0]

### LA GeoDF

In [ ]:
geometryMapping_la = [Point(xy) for xy in zip(los_angeles.lng, los_angeles.lat)]
geo_la = GeoDataFrame(los_angeles, crs={'init' :'epsg:4326'}, geometry=geometryMapping_la)
geo_la=geo_la.to_crs(california_crs)
print(type(geo_la))
print(geo_la.crs)

### SF GeoDF

In [ ]:
geometryMapping = [Point(xy) for xy in zip(sfbay.lng, sfbay.lat)]
geo_sf = GeoDataFrame(sfbay, crs={'init' :'epsg:4326'}, geometry=geometryMapping)
geo_sf=geo_sf.to_crs(california_crs)
print(type(geo_sf))
print(geo_sf.crs)

In [ ]:
la_joined = gpd.sjoin(geo_la, california, how='inner', op='intersects')

In [ ]:
sf_joined = gpd.sjoin(geo_sf, california, how="inner", op='intersects')

In [ ]:
# Dropping unnecessary fields that came from Census shapefile
la_joined.drop(['ALAND','LSAD','NAME','AWATER','AFFGEOID','STATEFP','COUNTYFP'], axis=1, inplace=True)
sf_joined.drop(['ALAND','LSAD','NAME','AWATER','AFFGEOID','STATEFP','COUNTYFP'], axis=1, inplace=True)

### Mapping

In [ ]:
base = california.plot(color='gray', linewidth=.1)
sf_joined.plot(color='red', ax=base)
la_joined.plot(color='b', ax=base)
plt.show()

### Comparing Vacant to Shared Listings

In [ ]:
vacant = pd.read_csv('vacant_april_listings.csv',dtype={'fips_block':str})

In [ ]:
# Creating a GEOID field that's comparable to shared listings
vacant['GEOID'] = vacant['fips_block'].str[:-4]

In [ ]:
# Turning studios into 1BRs for calculating rent per bedroom
vacant['beds2'] = vacant['beds']
vacant.loc[vacant.beds2 == 0, 'beds2'] = 1


In [ ]:
vacant['rent_per_br'] = vacant['rent']/vacant['beds2']

In [ ]:
# in this cell, define the values by which we will filter the 3 columns. This will vary depending on the sample we're looking at. 
upper_percentile = 0.99
lower_percentile = 0.01

# how many rows would be within the upper and lower percentiles?
upper = int(len(vacant) * upper_percentile)
lower = int(len(vacant) * lower_percentile)

# get the rent/sqft values at the upper and lower percentiles
rent_sqft_sorted = vacant['rent_sqft'].sort_values(ascending=True, inplace=False)
upper_rent_sqft = rent_sqft_sorted.iloc[upper]
lower_rent_sqft = rent_sqft_sorted.iloc[lower]

# get the rent values at the upper and lower percentiles
rent_sorted = vacant['rent'].sort_values(ascending=True, inplace=False)
upper_rent = rent_sorted.iloc[upper]
lower_rent = rent_sorted.iloc[lower]

# get the sqft values at the upper and lower percentiles
sqft_sorted = vacant['sqft'].sort_values(ascending=True, inplace=False)
upper_sqft = sqft_sorted.iloc[upper]
lower_sqft = sqft_sorted.iloc[lower]

print('valid rent_sqft range:', [lower_rent_sqft, upper_rent_sqft])
print('valid rent range:', [lower_rent, upper_rent])
print('valid sqft range:', [lower_sqft, upper_sqft])

In [ ]:
#Filtering out rows with unreasonable rent prices 
rent_mask = (vacant['rent'] > lower_rent) & (vacant['rent'] < upper_rent)

In [ ]:
filtered_vacant = pd.DataFrame(vacant[rent_mask])

In [ ]:
# Snapshot of median prices for total sample
print('median rent for all vacant listings:', filtered_vacant['rent'].median())
print('median rent for all vacant Studio Units:',(filtered_vacant[filtered_vacant['beds']==0])['rent_per_br'].median())
print('median rent for all vacant 1BR Units:',(filtered_vacant[filtered_vacant['beds']==1])['rent_per_br'].median())

print('median rent per bedroom for all vacant listings:', filtered_vacant['rent_per_br'].median())
print('median rent per bedroom for all multi-bedroom vacant listings:', (filtered_vacant[filtered_vacant['beds'] > 1])['rent_per_br'].median())

In [ ]:
vacant_sfbay = vacant[vacant['region']=='sfbay']

In [ ]:
vacant_sfbay.shape

In [ ]:
# Note: If we apply a filter for individual regions, we'd probably want to adjust the parameters to account for the smaller 
# sample size and different market; for example, $5000 is not a super atypical price in the Bay Area, shouldn't be filtered out 
print('median rent for all SF vacant listings:', vacant_sfbay['rent'].median())
print('median rent for all SF vacant Studio Units:',(vacant_sfbay[vacant_sfbay['beds']==0])['rent_per_br'].median())
print('median rent for all SF vacant 1BR Units:',(vacant_sfbay[vacant_sfbay['beds']==1])['rent_per_br'].median())

print('median rent per bedroom for all SF vacant listings:', vacant_sfbay['rent_per_br'].median())
print('median rent per bedroom for all SF multi-bedroom vacant listings:', (vacant_sfbay[vacant_sfbay['beds'] > 1])['rent_per_br'].median())

In [ ]:
print('median rent for room in SF shared listing:', sf_joined['rent'].median())

In [ ]:
vacant_la = vacant[vacant['region']=='losangeles']

In [ ]:
#vacant2 = vacant[(vacant['GEOID'] != '')]

In [ ]:
# Grouping and aggragating by GEOID for vacant SF listings
sf_vacant_grouped = vacant_sfbay.groupby(by='GEOID')
sf_vacant_grouped = sf_vacant_grouped.agg({'rent_per_br':['mean','median','count']}).reset_index()

In [ ]:
#Now, need to turn newly created multi-index dataframe into a single index dataframe 
sf_vacant_grouped.columns = sf_vacant_grouped.columns.get_level_values(1)

sf_vacant_grouped.columns = ['GEOID','tract_mean','tract_median','tract_count']

In [ ]:
sf_vacant_grouped.sort_values(by='tract_count', ascending=False)

In [ ]:
sf_shared_grouped = sf_joined.groupby(by='GEOID')
sf_shared_grouped = sf_shared_grouped.agg({'rent':['mean','median','count']}).reset_index()

In [ ]:
#Now, need to turn newly created multi-index 
sf_shared_grouped.columns = sf_shared_grouped.columns.get_level_values(1)

In [ ]:
sf_shared_grouped.columns = ['GEOID','tract_mean','tract_median','tract_count']

In [ ]:
sf_shared_grouped.sort_values(by='tract_count', ascending=False)

### SF Comparison

In [ ]:
# Merging shared and vacant listings on GEOID
sf_compare=pd.merge(sf_shared_grouped,sf_vacant_grouped,how='outer',on='GEOID')

In [ ]:
sf_compare.sort_values(by='tract_count_x', ascending=False).head()

In [ ]:
# Converting NaNs to 0s
#sf_compare = sf_compare.fillna(0)

In [ ]:
# Creating column that calculates difference between tract medians for vacant and shared listings
sf_compare['difference'] = sf_compare['tract_median_y'] - sf_compare['tract_median_x']

In [ ]:
# Sorting by tracts with most shared listings
sf_compare.sort_values(by='tract_count_x', ascending=False).head()

In [ ]:
# Necessary to keep lead '0' in GEOID column when exported to .csv. Only for ArcMap
# sf_compare.GEOID = sf_compare.GEOID.apply('="{}"'.format)

In [ ]:
sf_compare.to_csv('sf_compare.csv')

## Visualz

In [ ]:
listings_ratios['all_vacant_listings'] = vacant['region'].value_counts()
listings_ratios['shared_to_vacant_ratio'] = listings_ratios['deduplicated_pid']/listings_ratios['all_vacant_listings']

In [ ]:
listings_ratios.head()

In [ ]:
# plot the ratios of shared and vacant listings, by region
countdata = listings_ratios.sort_values(by='all_shared_listings', ascending=False)[['deduplicated_pid','all_vacant_listings']].head(20)
countdata.columns = ['Shared Listings', 'Vacant Listings']
ax = countdata.plot(kind='bar',
                    stacked=True,
                    figsize=[9, 6], 
                    width=0.6, 
                    alpha=0.5, 
                    color=['b','m'],
                    edgecolor='k',
                    grid=False)

ax.yaxis.grid(True)
ax.set_xticks(range(0, len(countdata)))
ax.set_xticklabels(countdata.index, rotation=40, rotation_mode='anchor', ha='right', fontproperties=ticks_font)
for label in ax.get_yticklabels():
        label.set_fontproperties(ticks_font)
ax.set_title('Unique and Duplicate Shared Rental Listings, by region', fontproperties=title_font)
ax.set_xlabel('', fontproperties=label_font)
ax.set_ylabel('Total number of listings', fontproperties=label_font)        

#save_fig(plt.gcf(), 'count_unique_duplicate_listings.png')
plt.show()